In [2]:
   %pip install seaborn
   %pip install statsmodels
   %pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.1 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 307 kB 46.5 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
#Load your dataset
df = pd.read_csv("/Users/edilbekabdyrakhmanov/Documents/GitHub/bakeryy/0_DataPreparation/initialdata/merged_data_temperature+holidays+weather_impressions.csv") # 

#Ensure the 'Datum' column is in datetime format
df['Datum'] = pd.to_datetime(df['Datum'])

#Define time ranges
train_start = '2013-07-01'
train_end = '2017-07-31'
val_start = '2017-08-01'
val_end = '2018-07-31'
test_start = '2018-08-01'
test_end = '2019-07-30'

#Split data
train_data = df[(df['Datum'] >= train_start) & (df['Datum'] <= train_end)]
validation_data = df[(df['Datum'] >= val_start) & (df['Datum'] <= val_end)]
test_data = df[(df['Datum'] >= test_start) & (df['Datum'] <= test_end)]

#Optional: print shapes
print("Train shape:", train_data.shape)
print("Validation shape:", validation_data.shape)
print("Test shape:", test_data.shape)

# Load and prepare the data (assuming your existing split)
def prepare_bakery_data(df):
    """
    Prepare bakery data by removing problematic variables and adding time-based features
    """
    # Make a copy to avoid modifying original
    data = df.copy()
    
    # Remove variables we decided to handle carefully
    columns_to_remove = ['Wettercode', 'AverageTemp', 'Temp_vs_Avg']
    existing_cols_to_remove = [col for col in columns_to_remove if col in data.columns]
    if existing_cols_to_remove:
        data = data.drop(columns=existing_cols_to_remove)
        print(f"Removed columns: {existing_cols_to_remove}")
    
    # Ensure Datum is datetime
    data['Datum'] = pd.to_datetime(data['Datum'])
    
    # Add time-based features
    data['Month'] = data['Datum'].dt.month
    data['Day_of_Year'] = data['Datum'].dt.dayofyear
    data['Week_of_Year'] = data['Datum'].dt.isocalendar().week
    data['Quarter'] = data['Datum'].dt.quarter
    data['Year'] = data['Datum'].dt.year
    
    # Add cyclical encoding for better temporal representation
    # Month (12 months cycle)
    data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
    data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)
    
    # Day of year (365 day cycle)
    data['Day_sin'] = np.sin(2 * np.pi * data['Day_of_Year'] / 365)
    data['Day_cos'] = np.cos(2 * np.pi * data['Day_of_Year'] / 365)
    
    # Week of year (52 week cycle)
    data['Week_sin'] = np.sin(2 * np.pi * data['Week_of_Year'] / 52)
    data['Week_cos'] = np.cos(2 * np.pi * data['Week_of_Year'] / 52)
    
    print("Added time-based features:")
    print("- Month, Day_of_Year, Week_of_Year, Quarter, Year")
    print("- Cyclical encodings: Month_sin/cos, Day_sin/cos, Week_sin/cos")
    
    return data

def encode_categorical_variables(data):
    """
    Encode categorical variables for modeling
    """
    # Make a copy
    data_encoded = data.copy()
    
    # Label encode categorical variables
    label_encoders = {}
    categorical_cols = ['Weekday', 'Weather_Impression']
    
    for col in categorical_cols:
        if col in data_encoded.columns:
            le = LabelEncoder()
            # Handle missing values
            data_encoded[col] = data_encoded[col].fillna('Unknown')
            data_encoded[f'{col}_encoded'] = le.fit_transform(data_encoded[col])
            label_encoders[col] = le
            print(f"Encoded {col}: {list(le.classes_)}")
    
    return data_encoded, label_encoders

def prepare_features_and_target(data):
    """
    Prepare feature matrix and target variable, handling missing values
    """
    # Define feature columns (excluding target and identifier columns)
    feature_cols = [
        'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 
        'Temp_Deviation', 'Is_Holiday', 'KielerWoche',
        'Month', 'Day_of_Year', 'Week_of_Year', 'Quarter', 'Year',
        'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Week_sin', 'Week_cos',
        'Weekday_encoded', 'Weather_Impression_encoded'
    ]
    
    # Filter only existing columns
    available_features = [col for col in feature_cols if col in data.columns]
    
    # Remove rows where target variable (Umsatz) is missing
    data_clean = data.dropna(subset=['Umsatz']).copy()
    
    # Prepare feature matrix
    X = data_clean[available_features].copy()
    y = data_clean['Umsatz'].copy()
    
    # Handle missing values in features
    # For numerical columns, use median
    numerical_cols = X.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        if X[col].isnull().any():
            median_val = X[col].median()
            X[col] = X[col].fillna(median_val)
            print(f"Filled {X[col].isnull().sum()} missing values in {col} with median: {median_val:.2f}")
    
    # For KielerWoche, fill with 0 (assuming it's binary)
    if 'KielerWoche' in X.columns:
        X['KielerWoche'] = X['KielerWoche'].fillna(0)
    
    print(f"Final feature matrix shape: {X.shape}")
    print(f"Target variable shape: {y.shape}")
    print(f"Features used: {list(X.columns)}")
    
    return X, y, data_clean

# Example usage with your existing data splits
def process_bakery_data(train_data, validation_data, test_data):
    """
    Process all data splits consistently
    """
    print("=== Processing Training Data ===")
    train_processed = prepare_bakery_data(train_data)
    train_encoded, label_encoders = encode_categorical_variables(train_processed)
    X_train, y_train, train_clean = prepare_features_and_target(train_encoded)
    
    print("\n=== Processing Validation Data ===")
    val_processed = prepare_bakery_data(validation_data)
    val_encoded, _ = encode_categorical_variables(val_processed)
    # Apply same encoders as training data
    for col, le in label_encoders.items():
        if col in val_encoded.columns:
            val_encoded[f'{col}_encoded'] = val_encoded[col].map(
                dict(zip(le.classes_, le.transform(le.classes_)))
            ).fillna(-1)  # Unknown categories get -1
    
    X_val, y_val, val_clean = prepare_features_and_target(val_encoded)
    
    print("\n=== Processing Test Data ===")
    test_processed = prepare_bakery_data(test_data)
    test_encoded, _ = encode_categorical_variables(test_processed)
    # Apply same encoders as training data
    for col, le in label_encoders.items():
        if col in test_encoded.columns:
            test_encoded[f'{col}_encoded'] = test_encoded[col].map(
                dict(zip(le.classes_, le.transform(le.classes_)))
            ).fillna(-1)  # Unknown categories get -1
    
    X_test, y_test, test_clean = prepare_features_and_target(test_encoded)
    
    return {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test,
        'label_encoders': label_encoders,
        'train_clean': train_clean,
        'val_clean': val_clean,
        'test_clean': test_clean
    }

# Simple regression models
def train_regression_models(X_train, y_train, X_val, y_val):
    """
    Train and evaluate regression models
    """
    models = {}
    results = {}
    
    # 1. Linear Regression
    print("Training Linear Regression...")
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    models['Linear Regression'] = lr
    
    # Predictions
    y_train_pred = lr.predict(X_train)
    y_val_pred = lr.predict(X_val)
    
    # Metrics
    results['Linear Regression'] = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_val_pred)),
        'train_r2': r2_score(y_train, y_train_pred),
        'val_r2': r2_score(y_val, y_val_pred),
        'train_mae': mean_absolute_error(y_train, y_train_pred),
        'val_mae': mean_absolute_error(y_val, y_val_pred)
    }
    
    # 2. Random Forest (for comparison)
    print("Training Random Forest...")
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    models['Random Forest'] = rf
    
    # Predictions
    y_train_pred_rf = rf.predict(X_train)
    y_val_pred_rf = rf.predict(X_val)
    
    # Metrics
    results['Random Forest'] = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred_rf)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_val_pred_rf)),
        'train_r2': r2_score(y_train, y_train_pred_rf),
        'val_r2': r2_score(y_val, y_val_pred_rf),
        'train_mae': mean_absolute_error(y_train, y_train_pred_rf),
        'val_mae': mean_absolute_error(y_val, y_val_pred_rf)
    }
    
    return models, results

def print_model_results(results):
    """
    Print model performance results in a nice format
    """
    print("\n" + "="*60)
    print("MODEL PERFORMANCE COMPARISON")
    print("="*60)
    
    for model_name, metrics in results.items():
        print(f"\n{model_name}:")
        print(f"  Training   - RMSE: {metrics['train_rmse']:.2f}, R²: {metrics['train_r2']:.3f}, MAE: {metrics['train_mae']:.2f}")
        print(f"  Validation - RMSE: {metrics['val_rmse']:.2f}, R²: {metrics['val_r2']:.3f}, MAE: {metrics['val_mae']:.2f}")

def analyze_feature_importance(model, feature_names, top_n=15):
    """
    Analyze feature importance for tree-based models
    """
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\nTop {top_n} Most Important Features:")
        print("-" * 40)
        for i, (_, row) in enumerate(importance_df.head(top_n).iterrows()):
            print(f"{i+1:2d}. {row['feature']:<25} {row['importance']:.4f}")
        
        return importance_df
    else:
        print("Model doesn't have feature importance attribute")
        return None

# Complete workflow function
def run_bakery_analysis(train_data, validation_data, test_data):
    """
    Run complete analysis workflow
    """
    print("Starting Bakery Sales Analysis...")
    print("="*50)
    
    # Process data
    processed_data = process_bakery_data(train_data, validation_data, test_data)
    
    # Train models
    models, results = train_regression_models(
        processed_data['X_train'], processed_data['y_train'],
        processed_data['X_val'], processed_data['y_val']
    )
    
    # Print results
    print_model_results(results)
    
    # Feature importance analysis
    if 'Random Forest' in models:
        importance_df = analyze_feature_importance(
            models['Random Forest'], 
            processed_data['X_train'].columns
        )
    
    return processed_data, models, results

# To run the complete analysis, use:
# processed_data, models, results = run_bakery_analysis(train_data, validation_data, test_data)
processed_data, models, results = run_bakery_analysis(train_data, validation_data, test_data)

Train shape: (7517, 15)
Validation shape: (1839, 15)
Test shape: (351, 15)
Starting Bakery Sales Analysis...
=== Processing Training Data ===
Removed columns: ['Wettercode', 'AverageTemp', 'Temp_vs_Avg']
Added time-based features:
- Month, Day_of_Year, Week_of_Year, Quarter, Year
- Cyclical encodings: Month_sin/cos, Day_sin/cos, Week_sin/cos
Encoded Weekday: ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
Encoded Weather_Impression: ['bad', 'good', 'okay', 'very bad', 'very good']
Filled 0 missing values in KielerWoche with median: 1.00
Final feature matrix shape: (7487, 19)
Target variable shape: (7487,)
Features used: ['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Temp_Deviation', 'Is_Holiday', 'KielerWoche', 'Month', 'Day_of_Year', 'Week_of_Year', 'Quarter', 'Year', 'Month_sin', 'Month_cos', 'Day_sin', 'Day_cos', 'Week_sin', 'Week_cos', 'Weekday_encoded', 'Weather_Impression_encoded']

=== Processing Validation Data ===
Removed columns: ['Wetterc

In [18]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

def prepare_ols_data(train_data):
    """
    Prepare data for OLS regression with enhanced features
    """
    # Make a copy
    data = train_data.copy()
    
    # Ensure Datum is datetime
    data['Datum'] = pd.to_datetime(data['Datum'])
    
    # Add time-based features (same as before)
    data['Month'] = data['Datum'].dt.month
    data['Day_of_Year'] = data['Datum'].dt.dayofyear
    data['Week_of_Year'] = data['Datum'].dt.isocalendar().week
    data['Quarter'] = data['Datum'].dt.quarter
    data['Year'] = data['Datum'].dt.year
    
    # Add cyclical encoding for better temporal representation
    data['Month_sin'] = np.sin(2 * np.pi * data['Month'] / 12)
    data['Month_cos'] = np.cos(2 * np.pi * data['Month'] / 12)
    data['Day_sin'] = np.sin(2 * np.pi * data['Day_of_Year'] / 365)
    data['Day_cos'] = np.cos(2 * np.pi * data['Day_of_Year'] / 365)
    data['Week_sin'] = np.sin(2 * np.pi * data['Week_of_Year'] / 52)
    data['Week_cos'] = np.cos(2 * np.pi * data['Week_of_Year'] / 52)
    
    # Fill missing values
    data['KielerWoche'] = data['KielerWoche'].fillna(0)
    data['Is_Holiday'] = data['Is_Holiday'].fillna(0)
    data['Temp_Deviation'] = data['Temp_Deviation'].fillna(data['Temp_Deviation'].median())
    data['Bewoelkung'] = data['Bewoelkung'].fillna(data['Bewoelkung'].median())
    data['Weather_Impression'] = data['Weather_Impression'].fillna('Unknown')
    
    return data

def build_ols_model(train_data):
    """
    Build OLS model with enhanced features following your teammate's approach
    """
    # Prepare the data
    data = prepare_ols_data(train_data)
    
    # Define target variable
    Y = data['Umsatz']
    
    # Build feature matrix step by step (similar to your teammate's approach)
    print("Building feature matrix...")
    
    # Start with product categories (Warengruppe) - same as teammate
    X_components = []
    
    # 1. Product categories (dummy variables)
    if 'Warengruppe' in data.columns:
        warengruppe_dummies = pd.get_dummies(data['Warengruppe'], 
                                           prefix='Warengruppe', 
                                           drop_first=True, 
                                           dtype=int)
        X_components.append(warengruppe_dummies)
        print(f"Added Warengruppe dummies: {list(warengruppe_dummies.columns)}")
    
    # 2. Weekday dummies - same as teammate
    weekday_dummies = pd.get_dummies(data['Weekday'], 
                                   prefix='Weekday', 
                                   drop_first=True, 
                                   dtype=int)
    X_components.append(weekday_dummies)
    print(f"Added Weekday dummies: {list(weekday_dummies.columns)}")
    
    # 3. Weather impression dummies (our addition)
    weather_dummies = pd.get_dummies(data['Weather_Impression'], 
                                   prefix='Weather', 
                                   drop_first=True, 
                                   dtype=int)
    X_components.append(weather_dummies)
    print(f"Added Weather dummies: {list(weather_dummies.columns)}")
    
    # 4. Continuous weather variables (enhanced from teammate's version)
    continuous_vars = ['Temperatur', 'Windgeschwindigkeit', 'Bewoelkung', 'Temp_Deviation']
    for var in continuous_vars:
        if var in data.columns:
            X_components.append(data[var])
            print(f"Added continuous variable: {var}")
    
    # 5. Binary variables
    binary_vars = ['Is_Holiday', 'KielerWoche']
    for var in binary_vars:
        if var in data.columns:
            X_components.append(data[var])
            print(f"Added binary variable: {var}")
    
    # 6. Time-based features (our major enhancement)
    time_features = ['Month', 'Quarter', 'Month_sin', 'Month_cos', 
                    'Day_sin', 'Day_cos', 'Week_sin', 'Week_cos']
    for var in time_features:
        X_components.append(data[var])
        print(f"Added time feature: {var}")
    
    # Combine all components
    X = pd.concat(X_components, axis=1)
    
    # Ensure all columns are numeric (float)
    X = X.apply(pd.to_numeric, errors='coerce')
    
    # Add constant (intercept) - same as teammate
    X = sm.add_constant(X)
    
    print(f"\nFinal feature matrix shape: {X.shape}")
    print(f"Features: {list(X.columns)}")
    
    # Clean data (remove NaN) - same approach as teammate
    valid_idx = Y.notna() & X.notna().all(axis=1)
    Y_clean = Y[valid_idx]
    X_clean = X[valid_idx]
    
    # Reset indices - same as teammate
    Y_clean.index = range(len(Y_clean))
    X_clean.index = range(len(X_clean))
    
    print(f"\nData after cleaning:")
    print(f"Observations: {len(Y_clean)}")
    print(f"Features: {X_clean.shape[1]} (including constant)")
    
    # Fit the OLS model - same as teammate
    print("\nFitting OLS model...")
    model = sm.OLS(Y_clean, X_clean)
    results = model.fit()
    
    return results, Y_clean, X_clean, data

def enhanced_ols_analysis(train_data):
    """
    Run the complete OLS analysis with enhanced features
    """
    print("="*60)
    print("ENHANCED OLS REGRESSION ANALYSIS")
    print("="*60)
    
    # Build and fit the model
    results, Y_clean, X_clean, processed_data = build_ols_model(train_data)
    
    # Print the summary (same as teammate)
    print("\nOLS REGRESSION RESULTS:")
    print("="*60)
    print(results.summary())
    
    # Additional diagnostics
    print("\n" + "="*60)
    print("ADDITIONAL MODEL DIAGNOSTICS")
    print("="*60)
    
    print(f"R-squared: {results.rsquared:.4f}")
    print(f"Adjusted R-squared: {results.rsquared_adj:.4f}")
    print(f"F-statistic: {results.fvalue:.2f}")
    print(f"F-statistic p-value: {results.f_pvalue:.2e}")
    print(f"AIC: {results.aic:.2f}")
    print(f"BIC: {results.bic:.2f}")
    
    # Most significant features
    print(f"\nMost Significant Features (p < 0.01):")
    print("-" * 50)
    significant_features = results.pvalues[results.pvalues < 0.01].sort_values()
    for feature, pval in significant_features.head(15).items():
        coef = results.params[feature]
        print(f"{feature:<30} Coef: {coef:8.3f}, p-value: {pval:.2e}")
    
    # Feature importance by absolute coefficient value
    print(f"\nLargest Coefficients (by absolute value):")
    print("-" * 50)
    abs_coefs = results.params.abs().sort_values(ascending=False)
    for feature in abs_coefs.head(15).index:
        if feature != 'const':  # Skip intercept
            coef = results.params[feature]
            pval = results.pvalues[feature]
            print(f"{feature:<30} Coef: {coef:8.3f}, p-value: {pval:.3f}")
    
    return results, Y_clean, X_clean, processed_data

# Run the analysis
print("Starting Enhanced OLS Analysis...")
results, Y_clean, X_clean, processed_data = enhanced_ols_analysis(train_data)

Starting Enhanced OLS Analysis...
ENHANCED OLS REGRESSION ANALYSIS
Building feature matrix...
Added Warengruppe dummies: ['Warengruppe_Brötchen', 'Warengruppe_Croissant', 'Warengruppe_Konditorei', 'Warengruppe_Kuchen', 'Warengruppe_Saisonbrot']
Added Weekday dummies: ['Weekday_Monday', 'Weekday_Saturday', 'Weekday_Sunday', 'Weekday_Thursday', 'Weekday_Tuesday', 'Weekday_Wednesday']
Added Weather dummies: ['Weather_good', 'Weather_okay', 'Weather_very bad', 'Weather_very good']
Added continuous variable: Temperatur
Added continuous variable: Windgeschwindigkeit
Added continuous variable: Bewoelkung
Added continuous variable: Temp_Deviation
Added binary variable: Is_Holiday
Added binary variable: KielerWoche
Added time feature: Month
Added time feature: Quarter
Added time feature: Month_sin
Added time feature: Month_cos
Added time feature: Day_sin
Added time feature: Day_cos
Added time feature: Week_sin
Added time feature: Week_cos

Final feature matrix shape: (7517, 30)
Features: ['cons

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).